In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
from sklearn import cross_validation
from sklearn.feature_extraction import DictVectorizer as DV
import matplotlib.pyplot as plt
from sklearn import svm, datasets

In [2]:
df = pd.read_csv('2008_100_300.csv', sep=',')
df.head()

,ID,Sex,Immun,CNS,Mediastinum,Age,Leuc,Leber,Milz,Zytogen,Region,Geb_month,Diag_month,height,weight,syndrome,Better
0,200,2,2,0,1,2.664002,4.4,2.5,0,1,42,7,1,90,13.8,0,100
1,210,1,2,0,1,4.394371,14.0,2.5,3,1221,104,11,1,114,18.5,0,100
2,500,2,13,0,1,6.880407,4.5,0.0,0,1,15,6,1,115,18.0,0,100
3,550,2,1,0,1,12.315190,2.8,4.0,0,1,26,1,1,46,157.0,0,0
4,560,1,2,0,1,2.611981,11.6,4.0,3,1,104,9,1,94,12.8,0,100


ID - Идентификационный номер пациента
Sex, Immun, CNS, Mediastinum, Zytogen, Region, Geb_month, Diag_month, syndrome - категориальные признаки
Age, Leuc, Leber, Milz, height, weight - количественные
Better - какое из двух видов лечения лучше. Если 0, значит, нельзя сказать, какое лучше.

Более подробно:
    Sex - пол (1-мужской, 2 - женский)
    Immun - имуннофенотип в закодированном виде
    CNS - статус центральной нервной системы(0 - не поражена, другое - разные варианты поражения)
    Mediastinum - аналогично CNS, но только для средостения
    Zytogen - наличие генетического нарушения (0 - нет, другое - код нарушения)
    Region - код региона, в котором пациент проходит лечение
    Geb_month - месяц  рождения пациента
    Diag_motnh - месяц постановки диагноза
    syndrome - наличия синдрома Дауна
    Age - возраст
    Leuc - число лейкоцитов в крови
    Leber - пальпируемый размер выступа печени из-под реберной дуги
    Milz - аналогично Leber для селезенки
    height, weight - рост, вес на момент постановки диагноза соответственно.


replace all nan values in column Zytogen with 1.

In [3]:
df['Zytogen'].fillna(1, inplace=True)

replace all nan values in column Region with 77.

In [4]:
df['Region'].fillna(77, inplace=True)

import data without nan to csv

In [5]:
df.to_csv('fnkc_without_nan.csv', sep=',', mode='w')

Converting categorical data into numbers

In [6]:
df_new_type = df
df_new_type = df_new_type.drop('ID', axis=1)
df_new_type['Age'] = pd.cut(df['Age'], 8, labels=[1, 2, 3, 4, 5, 6, 7, 8])
df_new_type['Leuc'] = pd.cut(df['Leuc'], 2, labels=[1, 2])
df_new_type['Leber'] = pd.cut(df['Leber'], 2, labels=[1, 2])
df_new_type['weight'] = pd.cut(df['weight'], 3, labels=[1, 2, 3])
df_new_type['height'] = pd.cut(df['height'], 3, labels=[1, 2, 3])

In [7]:
target = df_new_type.ix[:, [-1]]
df_new_type = df_new_type.drop('Better', axis=1)
column_names = df_new_type.columns.values
for col in column_names:
    df_new_type[col] = df_new_type[col].astype('category')
df_binary = pd.get_dummies(df_new_type)
df_binary['Better'] = pd.Series(target.values[:, 0], index=df_binary.index)
df_binary.to_csv('fnkc_binary.csv', sep=',', mode='w')

Divide data into two groups: test and train

In [8]:
test_df = df_binary.loc[df['Better'] == 0]
train_df = df_binary.loc[df['Better'] != 0]

features that will be used for training:

In [9]:
x = train_df.iloc[:, 0:20].values

target classes:

In [10]:
y = train_df.values[:, -1]

train model

In [11]:
clf = svm.SVC(kernel='rbf', gamma=0.7, C=1)

cross_val_score

In [12]:
scores = cross_validation.cross_val_score(clf, x, y, cv=10)

In [13]:
np.mean(scores)

0.80436314740112214

train model again

In [14]:
clf.fit(x, y)

SVC(C=1, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma=0.7, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

test features

In [15]:
features = train_df.iloc[:, 0:20].values

In [17]:
prediction = clf.predict(features)